# Demonstrace naší knihovny (Tým 12)

## Importy

Nejdříve přidáme složku s naší knihovnou do `sys.path`. Poté ji lze jednoduše importovat.

In [ ]:
import sys
import os
from os import path
import re
import unicodedata
import pandas as pd
import matplotlib.pyplot as plt

library_path = path.abspath(path.join(os.getcwd(), "../MH_decipher"))
if library_path not in sys.path:
    sys.path.append(library_path)

import mhdecipher # type: ignore

## Konstanty a pomocné funkce

In [ ]:
IMPORT_DIR_PATH = "./import"
EXPORT_DIR_PATH = "../export"

In [ ]:
def read_file(filepath: str) -> str:
    with open(filepath, "r", encoding="utf-8") as f:
        return f.read()

def write_file(filepath: str, data: str) -> str:
    with open(filepath, "w", encoding="utf-8") as f:
        return f.write(data)

def remove_diacritics(text: str) -> str:
    normalized_str = unicodedata.normalize("NFD", text)
    return "".join(c for c in normalized_str if unicodedata.category(c) != "Mn")

## Šifrování

In [ ]:
plaintext = "BYL_POZDNI_VECER_PRVNI_MAJ_VECERNI_MAJ_BYL_LASKY_CAS"
key = "DEFGHIJKLMNOPQRSTUVWXYZ_ABC"

ciphertext = mhdecipher.substitute_encrypt(plaintext, key)

print(ciphertext)

## Dešifrování

In [ ]:
ciphertext = "EAOCSRBGQLCYHFHUCSUYQLCPDMCYHFHUQLCPDMCEAOCODVNACFDV"
key = "DEFGHIJKLMNOPQRSTUVWXYZ_ABC"

plaintext = mhdecipher.substitute_decrypt(ciphertext, key)

print(plaintext)

## Získání bigramů

In [ ]:
bigrams = mhdecipher.get_bigrams("KRYPTOSYSTEM")
print(bigrams)

## Vytvoření teoretické bigramové matice

In [ ]:
ref_text = read_file("./Krakatit.txt")

# Extract only the actual text of the chapters
chapters_text = re.findall(r"Údaje o textu\n.*?\nPodtitulek: .*?\nO. Štorch-Marien 1924\n\n(.*?)\n\n\n\n\n\n", ref_text, re.DOTALL)

# Replace new lines in each chapter with a space
chapters_text = [s.replace("\n\n", " ") for s in chapters_text]

# Join the chapters into a single string
ref_text = "\n\n".join(chapters_text)

# Convert the text to our alphabet
ref_text = ref_text.upper()
ref_text = ref_text.replace(" ", "_")
ref_text = remove_diacritics(ref_text)

# Remove all characters that aren't in the alphabet and aren't a whitespace character
ref_text = re.sub(f"[^{mhdecipher.alphabet}\\s]", "", ref_text)

print(f"Using {len(ref_text)} characters of text to make a reference transition matrix.")

bigrams = mhdecipher.get_bigrams(ref_text)
TM_ref: pd.DataFrame = mhdecipher.transition_matrix(bigrams)

# Visualize the matrix
TM_ref.style.format().background_gradient(axis=None)

## Věrohodnost

Kód předpokládá vygenerovanou bigramovou matici z minulé sekce.

In [ ]:
plausibility = mhdecipher.plausibility("BYL_POZDNI_VECER_PRVNI_MAJ_VECERNI_MAJ_BYL_LASKY_CAS", TM_ref)
print(plausibility)

plausibility = mhdecipher.plausibility("EAOCSRBGQLCYHFHUCSUYQLCPDMCYHFHUQLCPDMCEAOCODVNACFDV", TM_ref)
print(plausibility)

Čitelný český text má větší věrohodnost než zašifrovaný/nečitelný text.

## Provedení kryptoanalýzy na zašifrovaném textu

Kód předpokládá vygenerovanou bigramovou matici z minulé sekce.

In [ ]:
ciphertext = read_file("./import/text_1000_sample_1_ciphertext.txt")
iter = 20_000
# start_key = "ABCDEFGHIJKLMNOPQRSTUVWXYZ_"
start_key = None  # the function will generate a random start key

key, decrypted_text, p = mhdecipher.prolom_substitute(ciphertext, TM_ref, iter, start_key)

print(f"Key: {key}")
print(f"Decrypted text: {decrypted_text}")
print(f"P: {p}")

## Provedení kryptoanalýzy na zašifrovaných textech ze zadání

In [ ]:
for filename in os.listdir(IMPORT_DIR_PATH):
    m = re.fullmatch("^(.*)_ciphertext.txt$", filename)
    if m is None:
        continue

    ciphertext_filepath = path.join(IMPORT_DIR_PATH, filename)
    plaintext_filepath = path.join(EXPORT_DIR_PATH, m.expand(r"\1_plaintext.txt"))
    key_filepath = path.join(EXPORT_DIR_PATH, m.expand(r"\1_key.txt"))

    if path.isfile(plaintext_filepath) or path.isfile(plaintext_filepath):
        print(f"Skipping {filename} because the plaintext or key already exists")
        continue

    ciphertext = read_file(ciphertext_filepath)
    iter = 20_000

    print(f"Cracking {filename}")
    key, decrypted_text, p = mhdecipher.prolom_substitute(ciphertext, TM_ref, iter)
    print()

    write_file(plaintext_filepath, decrypted_text)
    write_file(key_filepath, key)

## Analýza úspěšnosti prolomení

Analýza není jednoduchá, protože nemáme k dispozici správné klíče či originální texty. Všimli jsme si ovšem, že se jedná o úryvky z díla Krakatit. Budeme tedy uvažovat, že mezery v dešifrovaném textu našim algoritmem, jsou správně, a na jejich základě zkusíme najít stejné úseky v originálním díle.

In [ ]:
def similarity_score(str1: str, str2: str) -> int:
    if len(str1) != len(str2):
        raise ValueError("Lengths of strings don't match")
    
    l = len(str1)
    match_count = 0
    for i in range(l):
        if str1[i] == str2[i]:
            match_count += 1
    
    return match_count / l

fig, ax = plt.subplots(figsize=(10, 5))
ax.set_title("Similarity of decrypted texts to the original texts")

decrypted_plaintexts = []

# Load the decrypted texts
for filename in os.listdir(EXPORT_DIR_PATH):
    m = re.fullmatch("^text_(.*)_sample_(.*)_plaintext.txt$", filename)
    if m is None:
        continue
    
    plaintext_filepath = path.join(EXPORT_DIR_PATH, filename)
    decrypted_plaintext = read_file(plaintext_filepath)

    length = int(m.group(1))
    id = int(m.group(2))
    decrypted_plaintexts.append((length, id, decrypted_plaintext))

decrypted_plaintexts.sort()

labels = []
similarity_scores = []
colors = []

# Find the original plaintexts and calculate the similarity
for length, id, decrypted_plaintext in decrypted_plaintexts:
    label = f"text_{length}_sample_{id}"
    labels.append(label)

    pattern_words = [f"[A-Z]{{{len(c)}}}" for c in decrypted_plaintext.split("_")]
    pattern = "_".join(pattern_words)
    matches = re.findall(pattern, ref_text)
    if len(matches) == 0:
        print(f"Original plaintext for {label} was not found")
        similarity_scores.append(0)
        colors.append("red")
        continue
    if len(matches) > 1:
        print(f"More than 1 match was found for {label}")
        similarity_scores.append(0)
        colors.append("red")
        continue
    
    original_plaintext = matches[0]

    
    similarity_scores.append(similarity_score(decrypted_plaintext, original_plaintext))
    match length:
        case 250: colors.append("blue")
        case 500: colors.append("green")
        case 1000: colors.append("orange")
        case _: colors.append("red")


# Make a plot of similarities
ax.bar(labels, similarity_scores, color=colors)
ax.tick_params(axis="x", rotation=270)
ax.axhline(0.95, color="red")
plt.show()

Graf zobrazuje podobnost dešifrovaných textů a originálních textů. Různé délky šifrovaných textů jsou barevně rozlišeny. Červená přímka značí 95% podobnost.

Pokud se nepodařilo jednoznačně najít originální text, tak má sloupec podobnost 0.

Je vidět, že čím delší šifrovaný text máme, tím větší šance na úspěšné prolomení.

Nízká podobnost u krátkých textů není způsobena špatným prolomovacím algoritmem nebo nízkým počtem iterací, neboť tyto dešifrované texty dokonce mají vyšší hodnotu věrohodnosti než originální text. Krátký šifrovaný text zkrátka není vhodný pro tento typ kryptoanalýzy.